# Python Lab 1: Fourier Analysis and Spectral Visualization

**EE 451: Communications Systems**

**Name:** _________________________ **Date:** _____________

---

## Objectives

By completing this lab, you will:

1. Generate and visualize signals in both time and frequency domains
2. Apply the FFT to compute signal spectra
3. Observe time-frequency duality through pulse width variations
4. Measure signal bandwidth using null-to-null and 3-dB criteria

## Instructions

- Complete all code cells marked with `# TODO`
- Answer all questions in the markdown cells provided
- Run all cells and verify outputs before submission
- Submit this notebook (.ipynb) to Brightspace by the due date

---

## Setup

Run this cell to import required libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift

# Plotting settings
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 12

print("Setup complete!")

---

## Part 1: Generating and Visualizing Signals (20 points)

### Task 1.1: Create a Sinusoidal Signal

Generate a cosine signal with:
- Frequency: 5 Hz
- Amplitude: 2
- Duration: 1 second
- Sample rate: 1000 Hz

In [ ]:
# TODO: Define parameters
f0 = ___  # Frequency in Hz
A = ___   # Amplitude
fs = ___  # Sample rate in Hz
duration = ___  # Duration in seconds

# TODO: Create time vector
t = np.arange(0, duration, 1/fs)

# TODO: Generate cosine signal: x(t) = A * cos(2*pi*f0*t)
x = ___

# Plot the signal
plt.figure(figsize=(12, 4))
plt.plot(t, x, 'b-', linewidth=1)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'Cosine Signal: f = {f0} Hz, A = {A}')
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.5)
plt.show()

print(f"Number of samples: {len(t)}")
print(f"Sample spacing: {1/fs*1000:.2f} ms")

### Question 1.1 (5 points)

How many complete cycles of the cosine wave are visible in the first 0.5 seconds of the plot? Does this match the expected frequency?

**Your Answer:**

*[Write your answer here]*

### Task 1.2: Compute and Plot the Frequency Spectrum

Use the FFT to compute the magnitude spectrum of your cosine signal.

In [ ]:
# Compute FFT
N = len(x)
X = fft(x)
frequencies = fftfreq(N, 1/fs)

# Shift to center zero frequency
X_shifted = fftshift(X)
freq_shifted = fftshift(frequencies)

# Compute magnitude (normalize by N for amplitude)
magnitude = np.abs(X_shifted) / N

# TODO: Plot the magnitude spectrum
plt.figure(figsize=(12, 4))
plt.plot(freq_shifted, magnitude, 'r-', linewidth=1)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Magnitude Spectrum of Cosine Signal')
plt.grid(True, alpha=0.3)
plt.xlim(-20, 20)  # Zoom in around the signal frequency
plt.show()

# Find peak frequencies
peak_idx = np.where(magnitude > 0.1)[0]
print("Peak frequencies:", freq_shifted[peak_idx])
print("Peak magnitudes:", magnitude[peak_idx])

### Question 1.2 (5 points)

Why do you see TWO peaks in the spectrum (at +5 Hz and -5 Hz) instead of just one? What is the magnitude of each peak, and how does it relate to the original amplitude A=2?

**Your Answer:**

*[Write your answer here]*

---

## Part 2: Rectangular Pulse and Sinc Spectrum (25 points)

### Task 2.1: Create a Rectangular Pulse

Generate a rectangular pulse of width $\tau = 0.1$ seconds.

In [ ]:
# Parameters
fs = 1000  # Sample rate
tau = 0.1  # Pulse width in seconds
duration = 1.0

# Create centered time vector
t = np.linspace(-duration/2, duration/2, int(fs * duration))
dt = t[1] - t[0]

# TODO: Create rectangular pulse (1 when |t| < tau/2, else 0)
pulse = ___

# Plot the pulse
plt.figure(figsize=(12, 4))
plt.plot(t, pulse, 'b-', linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'Rectangular Pulse (τ = {tau} s)')
plt.grid(True, alpha=0.3)
plt.xlim(-0.3, 0.3)
plt.ylim(-0.1, 1.2)
plt.show()

### Task 2.2: Compute the Spectrum and Identify Nulls

In [ ]:
# Compute FFT
N = len(pulse)
Pulse_fft = fftshift(fft(pulse)) * dt  # Scale for continuous-time approximation
freq = fftshift(fftfreq(N, dt))

# Magnitude spectrum
magnitude = np.abs(Pulse_fft)

# Plot
plt.figure(figsize=(12, 5))
plt.plot(freq, magnitude, 'r-', linewidth=1.5)
plt.xlabel('Frequency (Hz)')
plt.ylabel('|X(f)|')
plt.title(f'Magnitude Spectrum of Rectangular Pulse (τ = {tau} s)')
plt.grid(True, alpha=0.3)
plt.xlim(-50, 50)

# TODO: Calculate theoretical first null frequency (hint: nulls occur at f = n/τ)
first_null = ___

# Mark the first nulls
plt.axvline(x=first_null, color='g', linestyle='--', linewidth=2, label=f'First null: ±{first_null} Hz')
plt.axvline(x=-first_null, color='g', linestyle='--', linewidth=2)
plt.legend()
plt.show()

print(f"Pulse width: τ = {tau} s")
print(f"First null frequency: 1/τ = {first_null} Hz")
print(f"Null-to-null bandwidth: 2/τ = {2*first_null} Hz")

### Question 2.1 (5 points)

The spectrum of a rectangular pulse is a sinc function. Looking at your plot:

a) Where are the first nulls located (in Hz)?

b) What is the null-to-null bandwidth?

c) How does the null-to-null bandwidth relate to the pulse width τ?

**Your Answer:**

*[Write your answer here]*

---

## Part 3: Time-Frequency Duality (25 points)

### Task 3.1: Vary Pulse Width and Observe Bandwidth Changes

Create pulses with different widths and compare their spectra.

In [ ]:
# TODO: Define three different pulse widths
tau_values = [0.05, 0.1, 0.2]  # Modify if desired
colors = ['blue', 'green', 'red']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for tau, color in zip(tau_values, colors):
    # Create pulse
    pulse = np.where(np.abs(t) < tau/2, 1, 0)
    
    # Compute spectrum
    Pulse_fft = fftshift(fft(pulse)) * dt
    magnitude = np.abs(Pulse_fft)
    
    # Plot time domain
    axes[0].plot(t, pulse, color=color, linewidth=2, label=f'τ = {tau} s')
    
    # Plot frequency domain
    axes[1].plot(freq, magnitude, color=color, linewidth=1.5, label=f'τ = {tau} s, BW = {2/tau:.0f} Hz')

axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Time Domain: Pulse Width Comparison')
axes[0].set_xlim(-0.3, 0.3)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('|X(f)|')
axes[1].set_title('Frequency Domain: Bandwidth Comparison')
axes[1].set_xlim(-50, 50)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Task 3.2: Calculate and Verify Time-Bandwidth Product

In [ ]:
# TODO: Complete the table
print("Time-Bandwidth Product Analysis")
print("=" * 60)
print(f"{'Pulse Width (τ)':<20} {'Bandwidth (BW)':<20} {'τ × BW':<20}")
print("-" * 60)

for tau in tau_values:
    bandwidth = ___  # TODO: Calculate null-to-null bandwidth
    product = ___    # TODO: Calculate time-bandwidth product
    print(f"{tau:<20.3f} {bandwidth:<20.1f} {product:<20.1f}")

print("=" * 60)

### Question 3.1 (10 points)

Based on your observations:

a) What happens to the bandwidth when you make the pulse narrower?

b) What happens to the bandwidth when you make the pulse wider?

c) Is the time-bandwidth product constant? What is its approximate value for the null-to-null bandwidth?

d) Why is time-frequency duality important for communication system design?

**Your Answer:**

*[Write your answer here]*

---

## Part 4: 3-dB Bandwidth Measurement (20 points)

### Task 4.1: Measure 3-dB Bandwidth

The 3-dB bandwidth is the frequency range where the power is at least half the maximum (or magnitude is at least $1/\sqrt{2}$ of maximum).

In [ ]:
# Create a Gaussian pulse (has well-defined 3-dB bandwidth)
sigma = 0.05  # Standard deviation
gaussian_pulse = np.exp(-t**2 / (2 * sigma**2))

# Compute spectrum
G_fft = fftshift(fft(gaussian_pulse)) * dt
G_magnitude = np.abs(G_fft)

# Normalize to find 3-dB point
G_normalized = G_magnitude / np.max(G_magnitude)

# TODO: Find 3-dB threshold (magnitude where power is half)
threshold_3dB = ___  # Hint: 1/sqrt(2) ≈ 0.707

# Find frequencies where magnitude crosses threshold
above_threshold = G_normalized > threshold_3dB
# Find the positive frequency where it drops below threshold
positive_freq_mask = freq > 0
crossings = np.where(above_threshold[positive_freq_mask] == False)[0]
if len(crossings) > 0:
    f_3dB = freq[positive_freq_mask][crossings[0]]
else:
    f_3dB = np.max(freq)

# Plot
plt.figure(figsize=(12, 5))
plt.plot(freq, G_normalized, 'b-', linewidth=2, label='Gaussian Spectrum')
plt.axhline(y=threshold_3dB, color='r', linestyle='--', linewidth=2, label=f'3-dB threshold ({threshold_3dB:.3f})')
plt.axvline(x=f_3dB, color='g', linestyle='--', linewidth=1.5, alpha=0.7)
plt.axvline(x=-f_3dB, color='g', linestyle='--', linewidth=1.5, alpha=0.7, label=f'3-dB frequencies (±{f_3dB:.1f} Hz)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Normalized Magnitude')
plt.title('3-dB Bandwidth Measurement')
plt.xlim(-30, 30)
plt.ylim(0, 1.1)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Gaussian pulse σ = {sigma} s")
print(f"3-dB half-bandwidth: f_3dB = {f_3dB:.1f} Hz")
print(f"3-dB full bandwidth: 2 × f_3dB = {2*f_3dB:.1f} Hz")

### Question 4.1 (10 points)

a) What is the 3-dB bandwidth of the Gaussian pulse with σ = 0.05 s?

b) Why is the 3-dB bandwidth different from the null-to-null bandwidth?

c) Which bandwidth definition would be more useful for: (i) designing a filter, (ii) calculating channel capacity?

**Your Answer:**

*[Write your answer here]*

---

## Part 5: Application - Sum of Sinusoids (10 points)

### Task 5.1: Create and Analyze a Multi-Frequency Signal

In [ ]:
# Create a signal with multiple frequency components
fs = 1000
duration = 1.0
t = np.arange(0, duration, 1/fs)

# TODO: Create signal with three components:
# - 5 Hz with amplitude 1.0
# - 12 Hz with amplitude 0.5
# - 20 Hz with amplitude 0.3
x = ___

# Compute spectrum
N = len(x)
X = fftshift(fft(x)) / N
freq = fftshift(fftfreq(N, 1/fs))
magnitude = np.abs(X)

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(t, x, 'b-', linewidth=1)
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Multi-Frequency Signal')
axes[0].set_xlim(0, 0.5)
axes[0].grid(True, alpha=0.3)

axes[1].stem(freq, magnitude, linefmt='r-', markerfmt='ro', basefmt='k-')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Magnitude')
axes[1].set_title('Magnitude Spectrum')
axes[1].set_xlim(-30, 30)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find and print peaks
peak_threshold = 0.1
peaks = np.where(magnitude > peak_threshold)[0]
print("\nDetected frequency components:")
for idx in peaks:
    if freq[idx] > 0:  # Only positive frequencies
        print(f"  f = {freq[idx]:.1f} Hz, amplitude = {2*magnitude[idx]:.2f}")

### Question 5.1 (10 points)

a) Do the peaks in the spectrum match the frequencies you used to create the signal?

b) Do the peak magnitudes match the original amplitudes? (Note: consider that the FFT shows both positive and negative frequencies)

c) How would this technique be useful for analyzing an unknown signal, such as an audio recording or radio transmission?

**Your Answer:**

*[Write your answer here]*

---

## Lab Summary

### Key Concepts Demonstrated

1. **FFT Analysis**: Converting time-domain signals to frequency domain
2. **Sinc Spectrum**: Rectangular pulse → sinc function in frequency
3. **Time-Frequency Duality**: Narrow pulse = wide spectrum, wide pulse = narrow spectrum
4. **Bandwidth Measurements**: Null-to-null vs. 3-dB bandwidth
5. **Spectral Analysis**: Identifying frequency components in complex signals

### Submission Checklist

- [ ] All TODO cells completed with working code
- [ ] All questions answered
- [ ] All cells executed (outputs visible)
- [ ] Notebook saved

---

*Submit this notebook to Brightspace by the due date.*